<a href="https://colab.research.google.com/github/lakshayanand96/Pharaphaser/blob/main/Paraphase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install simpletransformers==0.60.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [4]:
import warnings

import pandas as pd

In [5]:
%cd /content/drive/MyDrive/RephraseitDev

/content/drive/MyDrive/RephraseitDev


In [6]:
!ls

cache_dir		  outputs			 runs
dev.tsv			  parabank_5m.tsv		 test.tsv
msr_paraphrase_test.txt   quora_duplicate_questions.tsv  train.tsv
msr_paraphrase_train.txt  rephrase.ipynb		 wandb


In [7]:
def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", nrows=200, error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df
  

In [8]:
df = load_data (
    "quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


In [9]:
q_train, q_test = train_test_split(df)

In [10]:
print(q_train)

                                            input_text  \
165  Which is the best earphone with deep bass unde...   
31   What are some special cares for someone with a...   
62   How is the new Harry Potter book 'Harry Potter...   
120  Why my question was marked as needing imrovement?   
58   I was suddenly logged off Gmail. I can't remem...   
86   What were the major effects of the cambodia ea...   
159           Why nobody answer my questions in Quora?   
93                            What causes a nightmare?   
188  Is it possible to pursue many different things...   
73   How helpful is QuickBooks' auto data recovery ...   
12                What can make Physics easy to learn?   
193  What is the best combination of courses I can ...   
152               Why does Red keep the keys in OITNB?   
182  What if I hired two private eyes and ordered t...   
32   What Game of Thrones villain would be the most...   
198           How can I become more fluent in Chinese?   
173  Does anyo

In [11]:
print(q_test)

                                            input_text  \
16                        What does manipulation mean?   
92   What are some of the best romantic movies in E...   
175               Which is the best fixed income fund?   
122  What mineral holds the highest electrical charge?   
158  What does the Quora website look like to membe...   
13         What was your first sexual experience like?   
156  If I do not monetize YouTube videos & upload c...   
163         How do I use Twitter as a business source?   
144  Is Morgan Freeman correct when he says the onl...   
143                       Does a black hole have mass?   
136  Does it matter whether humans are selfish or e...   
49                              How do I make friends.   
100  Will there really be any war between India and...   
178       Why does some people prefer small families.?   
197  What are some must watch TV shows before you die?   
66                    What is the best book ever made?   
104   Can I re

In [12]:
 
def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [13]:
train_df = q_train[["prefix", "input_text", "target_text"]]
eval_df = q_test[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

In [14]:

model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 32
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 1000
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = True
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 4
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"

In [15]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda= False
)

In [16]:
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/54 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lakshayanand. Use `wandb login --relogin` to force relogin


Running Epoch 0 of 2:   0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

(28,
 {'global_step': [14, 28],
  'eval_loss': [1.7308357954025269, 1.7472107410430908],
  'train_loss': [1.380588412284851, 1.0298740863800049]})

In [17]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )